In [1]:
cd /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github

/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github


## Package installations

In [17]:
# To install required packages before training Dp-Promise
!pip install -r /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Obtaining opacus from git+https://github.com/deabfc/opacus_dp_promise.git@dp_promise#egg=opacus (from -r /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/requirements.txt (line 4))
  Cloning https://github.com/deabfc/opacus_dp_promise.git (to revision dp_promise) to ./src/opacus
  Running command git clone --filter=blob:none --quiet https://github.com/deabfc/opacus_dp_promise.git /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/src/opacus
  Running command git checkout -b dp_promise --track origin/dp_promise
  Switched to a new branch 'dp_promise'
  M     opacus/scripts/compute_dp_sgd_privacy.py
  M     scripts/install_via_pip.sh
  M     scripts/pytorch_install.sh
  M     website/scripts/build_website.sh
  M     website/sphinx/source/index.rst
  branch 'dp_promise' set up to track 'origin/dp_promise'.
  Resolved https://github.com/deabfc/opacus_dp_promise.git to commit 21f11051a1ac52ec888e135d7e58

## Preparing Data for DP-Promise Training

In [ ]:
# To prepare celeba dataset for training - 32x32 pixel conversion
!python prepare_celeba.py --source /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/img_align_celeba --dest /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32 --width 32 --height 32

In [ ]:
# To prepare celeba dataset for training - 64x64 pixel conversion
!python prepare_celeba.py --source /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/img_align_celeba --dest /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64 --width 64 --height 64

## Train and Test Data split

In [ ]:
# Splitting Train and Test data of celeba 32x32 for the DP-Promise model training
# First 80% was used as Training set and rest 20% was used as Test set(Held out as Non-Member for MIA attack)
import os
import shutil

# --- CONFIGURATION ---
source_dir = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32'   # Source folder
output_dir_1 = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train'  # 80% Train
output_dir_2 = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Test'   # 20% Test

# --- CREATE OUTPUT DIRECTORIES IF THEY DON'T EXIST ---
os.makedirs(output_dir_1, exist_ok=True)
os.makedirs(output_dir_2, exist_ok=True)

# --- GET SORTED FILE LIST ---
all_files = sorted([
    f for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
])

# --- CALCULATE SPLIT INDEX ---
split_index = int(0.8 * len(all_files))

# --- SPLIT FILES SEQUENTIALLY ---
files_A = all_files[:split_index]   # 80%
files_B = all_files[split_index:]   # 20%

# --- COPY FILES ---
for f in files_A:
    shutil.copy2(os.path.join(source_dir, f), os.path.join(output_dir_1, f))

for f in files_B:
    shutil.copy2(os.path.join(source_dir, f), os.path.join(output_dir_2, f))

print(f"✅ Done. {len(files_A)} files → {output_dir_1}, {len(files_B)} files → {output_dir_2}")


In [ ]:
# Splitting Train and Test data of celeba 64x64 for the DP-Promise model training
# First 80% was used as Training set and rest 20% was used as Test set(Held out as Non-Member for MIA attack)
import os
import shutil

# --- CONFIGURATION ---
source_dir = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64'   # Source folder
output_dir_1 = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train'  # 80% Train
output_dir_2 = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Test'   # 20% Test

# --- CREATE OUTPUT DIRECTORIES IF THEY DON'T EXIST ---
os.makedirs(output_dir_1, exist_ok=True)
os.makedirs(output_dir_2, exist_ok=True)

# --- GET SORTED FILE LIST ---
all_files = sorted([
    f for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
])

# --- CALCULATE SPLIT INDEX ---
split_index = int(0.8 * len(all_files))

# --- SPLIT FILES SEQUENTIALLY ---
files_A = all_files[:split_index]   # 80%
files_B = all_files[split_index:]   # 20%

# --- COPY FILES ---
for f in files_A:
    shutil.copy2(os.path.join(source_dir, f), os.path.join(output_dir_1, f))

for f in files_B:
    shutil.copy2(os.path.join(source_dir, f), os.path.join(output_dir_2, f))

print(f"✅ Done. {len(files_A)} files → {output_dir_1}, {len(files_B)} files → {output_dir_2}")


In [2]:
# checking image count for each category Member & Non-Member of both celeba dataset
import os
print("Total images 32:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32")))
print("Total images 32_Train:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train")))
print("Total images 32_Test:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Test")))
print("Total images 64:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64")))
print("Total images 64_Train:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train")))
print("Total images 64_Test:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Test")))

Total images 32: 202599
Total images 32_Train: 162079
Total images 32_Test: 40520
Total images 64: 202599
Total images 64_Train: 162079
Total images 64_Test: 40520


## Pretraining vanilla model for both celeba32 & celeba64

In [24]:
# Pretraining Vanilla model on celeba32 to give as already trained checkpoint for Dp-Promise
# Following the same process as in DP-Promise Git (But trained celeba instead of Imagenet, as we are not using Imagenet)
!python pretrain.py --config configs/vanilla/celeba_32/config.yaml --workdir checkpoints/vanilla --job-id Pretrain_celeba32

07/25/2025 02:27:32:INFO:Configuration: {'data': {'name': 'celeba', 'img_ch': 3, 'img_size': 32, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train', 'num_classes': 10, 'class_condition': False, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr': 0.0002, 'batch_size': 128, 'epochs': 50, 'ema_rate': 0.9999, 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 50, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}}
07/25/2025 02:27:32:INFO:Using seed: 42
07/25/2025 02:27:32:INFO:Load dataset celeba
07/25/2025 02:27:33:INFO:Num parameters: 35751939
07/25/2025 02:31:18:INFO:Epoch1: {'loss': 0.050669757319995054}
07/25/2025 02:34:12:INFO:Epoch2: {'loss': 0.03238056712999246}
07/25/2025 02:37:05:INFO:Epoch3: {'loss': 0.031280492045038216}
07/25/2025 02:39:57:INFO:Epoch4: {'loss': 0.03087975247744720

In [ ]:
# Pretraining Vanilla model on celeba64 to give as already trained checkpoint for Dp-Promise
# Following the same process as in DP-Promise Git (But trained celeba instead of Imagenet, as we are not using Imagenet)
!python pretrain.py --config configs/vanilla/celeba_64/config.yaml --workdir checkpoints/vanilla --job-id Pretrain_celeba64

## Dp-Promise Model Training & Synthesis Generation on Celeba32 with Epsilon 1, 5 and 10

In [2]:
# Code used for Model training on celeba 32 with Epsilon 1.0
!python train.py --config configs/dp_promise/celeba_32/eps1.0/config.yaml --workdir checkpoints/dp-promise --job-id Train_celeba32_Eps1.0

07/25/2025 07:04:43:INFO:Configuration: {'data': {'name': 'celeba', 'img_ch': 3, 'img_size': 32, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train', 'num_classes': 10, 'class_condition': True, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr1': 0.0003, 'lr2': 0.0003, 'batch_size1': 32, 'batch_size2': 4096, 'epochs1': 2, 'epochs2': 30, 'max_physical_batch_size': 16, 'num_noise_sample': 4, 'ema_rate': 0.9999, 'ckpt_path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/vanilla/vanilla_Pretrain_celeba32_celeba_0725-02:27:32/checkpoints/ema_50.ckpt', 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 200, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}, 'dp': {'max_grad_norm': 0.01, 'delta': 1e-06, 'epsilon': 1.0, 'S': 900, 'sigma': 3.66}}
07/25/2025 07:04:44:IN

In [3]:
# Code used for Model training on celeba 32 with Epsilon 5.0
!python train.py --config configs/dp_promise/celeba_32/eps5.0/config.yaml --workdir checkpoints/dp-promise --job-id Train_celeba32_Eps5.0

07/26/2025 22:50:56:INFO:Configuration: {'data': {'name': 'celeba', 'img_ch': 3, 'img_size': 32, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train', 'num_classes': 10, 'class_condition': True, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr1': 0.0003, 'lr2': 0.0003, 'batch_size1': 32, 'batch_size2': 4096, 'epochs1': 2, 'epochs2': 30, 'max_physical_batch_size': 16, 'num_noise_sample': 4, 'ema_rate': 0.9999, 'ckpt_path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/vanilla/vanilla_Pretrain_celeba32_celeba_0725-02:27:32/checkpoints/ema_50.ckpt', 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 200, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}, 'dp': {'max_grad_norm': 0.01, 'delta': 1e-06, 'epsilon': 5.0, 'S': 900, 'sigma': 1.01}}
07/26/2025 22:50:56:IN

In [4]:
# Code used for Model training on celeba 32 with Epsilon 10.0
!python train.py --config configs/dp_promise/celeba_32/eps10.0/config.yaml --workdir checkpoints/dp-promise --job-id Train_celeba32_Eps10.0

07/27/2025 19:01:27:INFO:Configuration: {'data': {'name': 'celeba', 'img_ch': 3, 'img_size': 32, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train', 'num_classes': 10, 'class_condition': True, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr1': 0.0003, 'lr2': 0.0003, 'batch_size1': 32, 'batch_size2': 4096, 'epochs1': 2, 'epochs2': 30, 'max_physical_batch_size': 16, 'num_noise_sample': 4, 'ema_rate': 0.9999, 'ckpt_path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/vanilla/vanilla_Pretrain_celeba32_celeba_0725-02:27:32/checkpoints/ema_50.ckpt', 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 200, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}, 'dp': {'max_grad_norm': 0.01, 'delta': 1e-06, 'epsilon': 10.0, 'S': 900, 'sigma': 0.73}}
07/27/2025 19:01:27:I

## Evaluation steps for Generated Image based on celeba32(Eps - 1, 5 & 10) - calculating IS & FID scores 

In [3]:
# compute dataset statistics for celeba 32
!python compute_dataset_stat.py --dataset celeba --path /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train

2025-07-30 02:22:17.172573: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 02:22:17.175131: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 02:22:17.206674: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 02:22:17.206915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-30 02:22:33.407566: W tensorflow/compiler/tf2t

In [4]:
# IS & FID Evaluation - result can be found under evaluation --> result --> celeba32 --> CelebA32_Eps1.0
!python eval_vision.py --dataset celeba --synthesis_path /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/dp-promise/dp-promise_Train_celeba32_Eps1.0_celeba_eps1.0_0725-07:04:43/evaluation/synthesis.npz --output_path results/celeba32/Eps1.0

2025-07-30 03:26:27.350852: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 03:26:27.353375: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 03:26:27.385750: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 03:26:27.386009: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-30 03:26:45.484619: W tensorflow/compiler/tf2t

In [5]:
# IS & FID Evaluation - result can be found under evaluation --> result --> celeba32 --> CelebA32_Eps5.0
!python eval_vision.py --dataset celeba --synthesis_path /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/dp-promise/dp-promise_Train_celeba32_Eps5.0_celeba_eps5.0_0726-22:50:56/evaluation/synthesis.npz --output_path results/celeba32/Eps5.0

2025-07-30 03:39:20.151589: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 03:39:20.154501: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 03:39:20.188003: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 03:39:20.188557: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-30 03:39:33.973843: W tensorflow/compiler/tf2t

In [6]:
# IS & FID Evaluation - result can be found under evaluation --> result --> celeba32 --> CelebA32_Eps10.0
!python eval_vision.py --dataset celeba --synthesis_path /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/dp-promise/dp-promise_Train_celeba32_Eps10.0_celeba_eps10.0_0727-19:01:27/evaluation/synthesis.npz --output_path results/celeba32/Eps10.0

2025-07-30 03:53:37.352370: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 03:53:38.034829: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 03:53:39.547100: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 03:53:39.547679: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-30 03:53:57.108580: W tensorflow/compiler/tf2t

## Dp-Promise Model Training & Synthesis Generation on Celeba64 with Epsilon 1, 5 and 10

In [9]:
# Code used for Model training on celeba 64 with Epsilon 1.0
!python train.py --config configs/dp_promise/celeba_64/eps1.0/config.yaml --workdir checkpoints/dp-promise --job-id Train_celeba64_Eps1.0

08/01/2025 03:38:12:INFO:Configuration: {'data': {'name': 'celeba64', 'img_ch': 3, 'img_size': 64, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train', 'num_classes': 10, 'class_condition': True, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr1': 0.0003, 'lr2': 0.0003, 'batch_size1': 16, 'batch_size2': 4096, 'epochs1': 1, 'epochs2': 15, 'max_physical_batch_size': 10, 'num_noise_sample': 4, 'ema_rate': 0.9999, 'ckpt_path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/vanilla/vanilla_Pretrain_celeba64_celeba64_0725-04:57:57/checkpoints/ema_50.ckpt', 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 200, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}, 'dp': {'max_grad_norm': 0.01, 'delta': 1e-06, 'epsilon': 1.0, 'S': 950, 'sigma': 2.83}}
08/01/2025 03:38:1

In [6]:
# Code used for Model training on celeba 64 with Epsilon 5.0
!python train.py --config configs/dp_promise/celeba_64/eps5.0/config.yaml --workdir checkpoints/dp-promise --job-id Train_celeba64_Eps5.0

08/09/2025 20:46:35:INFO:Configuration: {'data': {'name': 'celeba64', 'img_ch': 3, 'img_size': 64, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train', 'num_classes': 10, 'class_condition': True, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr1': 0.0003, 'lr2': 0.0003, 'batch_size1': 16, 'batch_size2': 4096, 'epochs1': 1, 'epochs2': 15, 'max_physical_batch_size': 10, 'num_noise_sample': 4, 'ema_rate': 0.9999, 'ckpt_path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/vanilla/vanilla_Pretrain_celeba64_celeba64_0725-04:57:57/checkpoints/ema_50.ckpt', 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 200, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}, 'dp': {'max_grad_norm': 0.01, 'delta': 1e-06, 'epsilon': 5.0, 'S': 950, 'sigma': 0.825}}
08/09/2025 20:46:

In [3]:
# Code used for Model training on celeba 64 with Epsilon 10.0
!python train.py --config configs/dp_promise/celeba_64/eps10.0/config.yaml --workdir checkpoints/dp-promise --job-id Train_celeba64_Eps10.0

08/09/2025 20:46:35:INFO:Configuration: {'data': {'name': 'celeba64', 'img_ch': 3, 'img_size': 64, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train', 'num_classes': 10, 'class_condition': True, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr1': 0.0003, 'lr2': 0.0003, 'batch_size1': 16, 'batch_size2': 4096, 'epochs1': 1, 'epochs2': 15, 'max_physical_batch_size': 10, 'num_noise_sample': 4, 'ema_rate': 0.9999, 'ckpt_path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/vanilla/vanilla_Pretrain_celeba64_celeba64_0725-04:57:57/checkpoints/ema_50.ckpt', 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 200, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}, 'dp': {'max_grad_norm': 0.01, 'delta': 1e-06, 'epsilon': 10.0, 'S': 950, 'sigma': 0.632}}
08/09/2025 20:46

## Evaluation steps for Geenrated Image based on celeba64(Eps - 1, 5 & 10) - calculating IS & FID scores

In [2]:
# compute dataset statistics for celeba 64
!python compute_dataset_stat.py --dataset celeba64 --path /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train

2025-07-30 01:22:21.648047: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 01:22:22.546823: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 01:22:23.960410: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 01:22:23.960792: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-30 01:22:36.292495: W tensorflow/compiler/tf2t

In [9]:
# IS & FID Evaluation - result can be found under evaluation --> result --> celeba64 --> CelebA64_Eps1.0
!python eval_vision.py --dataset celeba64 --synthesis_path /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/dp-promise/dp-promise_Train_celeba64_Eps1.0_celeba64_eps1.0_0803-05:32:50/evaluation/synthesis.npz --output_path results/celeba64/Eps1.0

2025-08-04 15:22:30.994531: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-04 15:22:31.745676: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-04 15:22:32.777729: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-04 15:22:32.778052: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-04 15:22:50.815177: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [8]:
# IS & FID Evaluation - result can be found under evaluation --> result --> celeba64 --> CelebA64_Eps5.0
!python eval_vision.py --dataset celeba64 --synthesis_path /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/dp-promise/dp-promise_Train_celeba64_Eps5.0_celeba64_eps5.0_0809-20:46:35/evaluation/synthesis.npz --output_path results/celeba64/Eps5.0

2025-08-11 06:18:02.728691: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 06:18:03.438464: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-11 06:18:04.382781: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-11 06:18:04.383344: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-11 06:18:19.466048: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [6]:
# IS & FID Evaluation - result can be found under evaluation --> result --> celeba64 --> CelebA64_Eps10.0
!python eval_vision.py --dataset celeba64 --synthesis_path /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/dp-promise/dp-promise_Train_celeba64_Eps10.0_celeba64_eps10.0_0809-20:46:35/evaluation/synthesis.npz --output_path results/celeba64/Eps10.0

2025-08-11 05:26:02.028728: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 05:26:02.626077: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-11 05:26:03.449669: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-11 05:26:03.449999: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-11 05:26:17.392551: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co